In [25]:
import gensim


def read_corpus(reviews, tokens_only=False):
    for i, line in enumerate(reviews):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])


In [26]:
import gzip
f = gzip.open("reviews_Beauty_5.json.gz")
reviews = []

for l in f:
    d = eval(l)
    reviews.append(d['reviewText'])
    if len(reviews) == 50000:
        break


train_corpus = list(read_corpus(reviews[:30000]))
test_corpus = list(read_corpus(reviews[30000:], tokens_only=True))

In [27]:
from gensim.models import Doc2Vec

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)
model.build_vocab(train_corpus) 
model.train(train_corpus, total_examples=model.corpus_count,epochs=model.epochs)



In [28]:
#需要安装 pip install testfixtures

inferred_vector = model.infer_vector(train_corpus[0].words)
sims = model.docvecs.most_similar([inferred_vector])
print(sims)

print(reviews[0])
print(reviews[21206])
print(reviews[28561])
print(reviews[5168])
print(reviews[18800])
print(reviews[14045])
print(reviews[3680])
print(reviews[6590])
print(reviews[27614])
print(reviews[16665])

[(0, 0.9664036631584167), (5496, 0.654357373714447), (26482, 0.650442361831665), (15571, 0.6437737345695496), (25624, 0.6262351870536804), (28961, 0.6235198974609375), (21556, 0.6199095249176025), (14045, 0.6128912568092346), (7581, 0.6115442514419556), (17174, 0.6082520484924316)]
Very oily and creamy. Not at all what I expected... ordered this to try to highlight and contour and it just looked awful!!! Plus, took FOREVER to arrive.
I HAVE ORDERED THIS MANY TIMES AND JUST LOVE IT.  I WOUL.D ALSO HIGHLY RECOMMEND IT TO OTHERS, GREAT PRODUCT
Just as expected
i have been using for about 13 years I always get compliments. love it. buy it. great investment. great everyday color.
Perfect for dirty blonde hair, love, love, love it!! Will keep continuing to buy when I'm out of what I have now.
The title of the review tells it all, it is by far the best self tanner I have ever used. Not orange not streaky just perfect.
I love Olay - I try other more expensive products and I go back to Olay.
Th

C:\Users\guany\AppData\Local\Temp\ipykernel_32184\2445987358.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims = model.docvecs.most_similar([inferred_vector])


In [29]:
models = [
    # PV-DBOW (Skip-Gram equivalent of Word2Vec)
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=10, epochs=50),
    
    # PV-DM w/average (CBOW equivalent of Word2Vec)
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=10, epochs =50),
]

documents = train_corpus
models[0].build_vocab(documents)
models[1].reset_from(models[0])

for model in models:
   model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)



In [30]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec((models[0], models[1]))

# model 1
inferred_vector =new_model.models[0].infer_vector(train_corpus[0].words)
sims2 = new_model.models[0].docvecs.most_similar([inferred_vector])
print(sims2)
# model 2
inferred_vector =new_model.models[1].infer_vector(train_corpus[0].words)
sims3 = new_model.models[1].docvecs.most_similar([inferred_vector])
print(sims3)


[(0, 0.8953219652175903), (6804, 0.4646955728530884), (28869, 0.46257513761520386), (3050, 0.4533193111419678), (20757, 0.4519912898540497), (17881, 0.4288741648197174), (1462, 0.42322108149528503), (8070, 0.41500720381736755), (28961, 0.409655898809433), (28010, 0.4063774347305298)]
[(0, 0.9050559401512146), (14045, 0.42165786027908325), (3680, 0.41847890615463257), (5970, 0.4160499572753906), (28561, 0.4142948389053345), (16315, 0.4098333418369293), (6590, 0.4080885350704193), (7413, 0.4073047637939453), (21206, 0.4066210389137268), (27614, 0.4052833914756775)]


C:\Users\guany\AppData\Local\Temp\ipykernel_32184\3960372065.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims2 = new_model.models[0].docvecs.most_similar([inferred_vector])
C:\Users\guany\AppData\Local\Temp\ipykernel_32184\3960372065.py:10: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  sims3 = new_model.models[1].docvecs.most_similar([inferred_vector])
